# CSE 255 Programming Assignment 7 - Data Analysis using XGBoost
 

## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found [here](Data_Processing_Whales.ipynb). The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer the <> notebook for the XGBoost Analysis.

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



## Notebook Setup

### Importing Required Libraries

In [ ]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle

### Loading Data

In [ ]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

## Helper functions

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [ ]:
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 0     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

### Computing average ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.11211652,  0.1430822 ,  0.1912412 ]),
 array([-0.24288348, -0.1580822 ,  0.0137588 ]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [ ]:
def calc_stats(margin_scores):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [ ]:
assert (min_score == np.array([-0.24, -0.16,  0.01])).all(), "Incorrect return value"
assert (max_score == np.array([-0.11,  0.14,  0.19])).all(), "Incorrect return value"

In [ ]:
# Hidden Tests Here

### Calculating predictions

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict -1
3. If respective minimum score and maximum score values are greater than 0, predict 1

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.49037403 -0.39128804 -0.33350122 -0.25335822 -0.20072483 -0.10861206
 -0.04009901  0.10376799  0.29661256  0.50962585]

max_scr (numpy array) = [-0.7791341  -0.68435764 -0.6014159  -0.53394866 -0.46818763 -0.4171182
 -0.31703883 -0.20936123 -0.06620912  0.22086589]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [ ]:
def predict(min_scr, max_scr):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
min_s = np.array([-0.4903740, -0.39128804, -0.33350122, -0.25335822, -0.20072483, -0.10861206, -0.04009901,\
                  0.10376799, 0.29661256, 0.50962585])
max_s = np.array([-0.7791341, -0.68435764, -0.6014159, -0.53394866, -0.46818763, -0.4171182,\
                 -0.31703883, -0.20936123, -0.06620912, 0.22086589])
pred = predict(min_s, max_s)
assert type(pred) == np.ndarray, 'Incorrect return type'
assert np.sum(np.equal(pred, np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1]))) == min_s.shape[0], 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      
#### Procedure

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API
* Sort the list of scores and normalize them by dividing them with the normalizing factor as max(scores) - min(scores)

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
 

In [ ]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size=3000, \
                   num_round=100, plst=xgboost_plst()):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def process(X_train, X_test, y_train, y_test):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=200, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

In [ ]:
sample_indices = np.load('Data/visible_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/visible_midpt.npy')

min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
assert (min_scr <= midpt).all(), "Incorrect range (mean - 2*std) to (mean + 2*std)"
assert (max_scr >= midpt).all(), "Incorrect range (mean - 2*std) to (mean + 2*std)"
length = max_scr - min_scr
avg_length = np.load('Data/visible_avg_length.npy')
assert (length < 2*avg_length).all(), "Incorrect length of range (mean - 2*std) to (mean + 2*std)"

In [ ]:
# Hidden Tests here

In [ ]:
# Hidden Tests here

# Submission Instructions

1. Validate your submission by submitting your assignment on **"Homework 7 Test Submission"** under **"Week 7-8"** on edX. This is worth 10 points.
2. Submit the final copy of your assignment on **"Homework 7 Final Submission"** under **"Week 7-8"** on edX. This will be evaluated for visible and hidden tests and is worth 90 points.